In [1]:
import os
from datetime import datetime, timedelta, timezone
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, get_sun, get_body
import astropy.units as u
from astropy.time import Time
import numpy as np

In [2]:
t_obs = '2024-01-24T03:24:11'
lat = 50.25
lon = 250.25
get_sun(Time(t_obs))

<SkyCoord (GCRS: obstime=2024-01-24T03:24:11.000, obsgeoloc=(0., 0., 0.) m, obsgeovel=(0., 0., 0.) m / s): (ra, dec, distance) in (deg, deg, AU)
    (305.57948987, -19.42058203, 0.98428231)>

In [4]:
def convert_to_altaz(ra, dec, location, obstime: str) -> tuple:
    """
        Converts ra and dec to alt az coordinates for a given location
        and observation time.
        
        In:
        ra: right ascension
        dec: declination
        location: lat lon location from EarthLocation
        obstime: str - format YYYY-MM-DDTHH:MM:SS
        
        Out:
        alt_deg - altitude in degrees
        az_deg - azimuth in degrees
    """
    # Convert RA and Dec from masked columns to arrays
    try:
        ra_str = ra.filled().data.tolist()
        dec_str = dec.filled().data.tolist()

        ra_deg = []
        dec_deg = []

        # Convert RA and Dec strings to degrees
        for ra_val, dec_val in zip(ra_str, dec_str):
            ra_components = ra_val.split()
            ra_hour, ra_min, ra_sec = map(float, ra_components)

            dec_components = dec_val.split()
            dec_deg_val, dec_min, dec_sec = map(float, dec_components)

            ra_deg.append((ra_hour + ra_min / 60 + ra_sec / 3600) * 15)  # Convert RA to degrees
            dec_deg.append(dec_deg_val + dec_min / 60 + dec_sec / 3600)  # Convert Dec to degrees
    except:
        ra_deg = ra
        dec_deg = dec
        
    # Convert time to the required format
    try:
        obstime = obstime.split('T')[0] + ' ' + obstime.split('T')[1]
    except:
        None # already in the correct format
    
    # Create SkyCoord object
    coord = SkyCoord(ra=ra_deg*u.deg, dec=dec_deg*u.deg, frame='icrs')
   
  # Convert to Alt-Az coordinates
    altaz = coord.transform_to(AltAz(obstime=obstime, location=location))
    
    # Extract Alt-Az coordinates
    alt_deg = altaz.alt.degree
    az_deg = altaz.az.degree
    
    return alt_deg, az_deg

def time_proc(obstime: str):
    """
        Builds time arrays for use in plotting object locations over time

        In:
        obstime: str
        
        Out:
        t_current: curent time as a Time object
        times_str: iso version of times
    """
    # Calculate half day times
    half_day = 0.5*u.day
    t_current = Time(obstime)
    t_start = t_current - half_day
    t_end = t_current + half_day
    
    # Generate an array of times within the specified range
    times = np.linspace(t_start.jd, t_end.jd, 1000)
    t_range = Time(times, format='jd', scale='utc')
    
    # Also create and array as a string version
    times_str = t_range.iso
    
    return t_current, times_str

In [5]:
# Function to get the ecliptic path
def ecliptic_pos(obstime: str, location: tuple):
    
    # Get the time arrays for the past 24 hours
    t_current, times_str = time_proc(obstime)
    
    print(t_current, times_str)
    
    # Get the sun's position at the current_time
    current_pos = get_sun(t_current)
    sun_ra_deg = current_pos.ra.deg
    sun_dec_deg = current_pos.dec.deg
    
    print(current_pos)
    
    # Convert to Alt-Az for current sun position
    sun_alt, sun_az = convert_to_altaz(sun_ra_deg, sun_dec_deg, location, obstime)
    sun_coords = (sun_az, 90 - sun_alt)
 
    # Convert to Alt-Az for all the times
    ecliptic_coords = ([], []) # AZ, ALT
    for _, tobs in enumerate(times_str):
        ecliptic_alt, ecliptic_az = convert_to_altaz(sun_ra_deg, sun_dec_deg, location,tobs)
        ecliptic_coords[0].append(ecliptic_az)
        ecliptic_coords[1].append(90 - ecliptic_alt)
    
    return sun_coords, ecliptic_coords

In [6]:
location = EarthLocation(lat=lat*u.deg, lon=lon*u.deg)

In [7]:
ecliptic_pos(obstime=t_obs, location=location)

2024-01-24T03:24:11.000 ['2024-01-23 15:24:11.000' '2024-01-23 15:25:37.486'
 '2024-01-23 15:27:03.973' '2024-01-23 15:28:30.459'
 '2024-01-23 15:29:56.946' '2024-01-23 15:31:23.432'
 '2024-01-23 15:32:49.919' '2024-01-23 15:34:16.405'
 '2024-01-23 15:35:42.892' '2024-01-23 15:37:09.378'
 '2024-01-23 15:38:35.865' '2024-01-23 15:40:02.351'
 '2024-01-23 15:41:28.838' '2024-01-23 15:42:55.324'
 '2024-01-23 15:44:21.811' '2024-01-23 15:45:48.297'
 '2024-01-23 15:47:14.784' '2024-01-23 15:48:41.270'
 '2024-01-23 15:50:07.757' '2024-01-23 15:51:34.243'
 '2024-01-23 15:53:00.730' '2024-01-23 15:54:27.216'
 '2024-01-23 15:55:53.703' '2024-01-23 15:57:20.189'
 '2024-01-23 15:58:46.676' '2024-01-23 16:00:13.162'
 '2024-01-23 16:01:39.649' '2024-01-23 16:03:06.135'
 '2024-01-23 16:04:32.622' '2024-01-23 16:05:59.108'
 '2024-01-23 16:07:25.595' '2024-01-23 16:08:52.081'
 '2024-01-23 16:10:18.568' '2024-01-23 16:11:45.054'
 '2024-01-23 16:13:11.541' '2024-01-23 16:14:38.027'
 '2024-01-23 16:16:04.

TypeError: concatenate() got an unexpected keyword argument 'dtype'